# Crowdsourcing for Information Retrieval: Tutorial at ECIR '23

![Crowd-Kit](https://repository-images.githubusercontent.com/343581364/50c743cc-9bc8-4bae-947c-7803158fb97e)

In this part of our tutorial, we will use the [Crowd-Kit](https://github.com/Toloka/crowd-kit) library for Python to aggregate and learn from crowd annotations.

We will be using the [CLINC150](https://paperswithcode.com/dataset/clinc150) dataset for evaluating the performance of *intent classification* systems. As this is a multi-class classification problem with 9 classes, we will use macro-averaged $F_1$ as the evaluation criterion.

**Outline:**

1. Install dependencies, load annotated and ground truth datasets, `train` and `test`.
2. Run traditional answer aggregation methods, such as [Majority Vote](https://toloka.ai/docs/crowd-kit/reference/crowdkit.aggregation.classification.majority_vote.MajorityVote) (MV), [Dawid-Skene](https://toloka.ai/docs/crowd-kit/reference/crowdkit.aggregation.classification.dawid_skene.DawidSkene) (DS), [Generative Models of Labels, Abilities, and Difficulties](https://toloka.ai/docs/crowd-kit/reference/crowdkit.aggregation.classification.glad.GLAD) (GLAD).
3. Choose the aggregation method on `train` subset and evaluate it on `test`.
4. Train a model on the aggregated dataset.
5. Train a model on the raw (non-aggregated) dataset.
6. Train a model on the raw dataset with [CrowdLayer](https://toloka.ai/docs/crowd-kit/reference/crowdkit.learning.crowd_layer.CrowdLayer) and [CoNAL](https://toloka.ai/docs/crowd-kit/reference/crowdkit.learning.conal.CoNAL).

# Answer Aggregation

We will install and import all the necessary libraries, which are [Crowd-Kit](https://github.com/Toloka/crowd-kit), [PyTorch Lightning](https://lightning.ai/pytorch-lightning), and [Sentence Transformers](https://sbert.net/).

In [ ]:
%%capture
%pip install -U crowd-kit lightning sentence-transformers

In [ ]:
import json

import lightning as L
import pandas as pd
import torch
import torchmetrics
from crowdkit.aggregation import DawidSkene, GLAD, MajorityVote
from crowdkit.learning import CoNAL, CrowdLayer
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from torch import nn, utils
from torch.utils.data import Dataset

Let's open the crowdsourced datasets with labeled train subset of our data (`labeled_train_data.tsv`) and test subset (`labeled_test_data.tsv`). We will re-arrange our dataset so its column names match what Crowd-Kit expects: `task`, `worker`, and `label`.

In [ ]:
def load_dataframe(filename):
    df_crowd = pd.read_csv(filename, sep='\t')

    # golden tasks are from 'val' dataset
    df_crowd = df_crowd[df_crowd['GOLDEN:intent'].isna()]

    df_crowd = df_crowd.rename(columns={
        'INPUT:query': 'task',
        'ASSIGNMENT:worker_id': 'worker',
        'OUTPUT:intent': 'label',
    })

    df_crowd = df_crowd[['task', 'worker', 'label']].reset_index(drop=True)

    return df_crowd

df_crowd_train = load_dataframe('labeled_train_data.tsv')
df_crowd_test = load_dataframe('labeled_test_data.tsv')

df_crowd_train

,task,worker,label
0,please show me a recipe for chili,b5329f9413a8795bfc3d45ffbbb1d31a,recipe
1,do i need to protect myself with some shots fo...,b5329f9413a8795bfc3d45ffbbb1d31a,vaccines
2,i would love to know the exchange rate between...,b5329f9413a8795bfc3d45ffbbb1d31a,exchange_rate
3,is it recommended to get any specific shots be...,b5329f9413a8795bfc3d45ffbbb1d31a,vaccines
4,what are some fun things i can partake in in a...,b5329f9413a8795bfc3d45ffbbb1d31a,travel_suggestion
...,...,...,...
1345,what's a good place to travel to,193f02fc8190d97758d5634cfd5168dc,travel_suggestion
1346,where should i spend my time off,193f02fc8190d97758d5634cfd5168dc,travel_suggestion
1347,do i need shots before i get to africa,193f02fc8190d97758d5634cfd5168dc,vaccines
1348,i want to know the nutrition info for chicken ...,193f02fc8190d97758d5634cfd5168dc,nutrition_info


Now let's download and open the ground truth dataset.

In [ ]:
!curl -sLO https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_small.json

In [ ]:
intents = {
    'restaurant_reviews',
    'restaurant_reservation',
    'nutrition_info',
    'recipe',
    'book_hotel',
    'timezone',
    'travel_suggestion',
    'exchange_rate',
    'vaccines',
}

with open('data_small.json') as f:
    oos = json.load(f)

In [ ]:
df_train = pd.DataFrame(oos['train'], columns=['query', 'intent'])

df_train = df_train[df_train['intent'].isin(intents)].reset_index(drop=True)

df_train

,query,intent
0,i need a blackberry pie recipe,recipe
1,find a recipe for german chocolate cake,recipe
2,please find me a recipe for spaghetti now,recipe
3,can you find me a recipe for sugar cookies,recipe
4,i need a recipe on how to make beef stew,recipe
...,...,...
445,what timezone is los angeles in,timezone
446,what timezone is new york in,timezone
447,reno's timezone is what,timezone
448,what timezone is china in,timezone


In [ ]:
df_test = pd.DataFrame(oos['test'], columns=['query', 'intent'])

df_test = df_test[df_test['intent'].isin(intents)].reset_index(drop=True)

df_test

,query,intent
0,"i'd like to make a reservation at rooth chris,...",restaurant_reservation
1,make a reservation for chik-fil-a at 3 o' cloc...,restaurant_reservation
2,reserve a table for 3 at 7 for olive garden,restaurant_reservation
3,will you reserve a table at olive garden for 3...,restaurant_reservation
4,"at 7, i need a table for 3 at olive garden",restaurant_reservation
...,...,...
265,i need a pasta recipe,recipe
266,i want a recipe for roasted veggies,recipe
267,what is in a burrito recipe,recipe
268,give me a tuna salad recipe,recipe


## Majority Vote

As we have multiple labels per each task, we will *aggregate* the labels to get a single label per task. We will start with the simplest self-explanatory heuristic, which is called Majority Vote (MV).

In [ ]:
agg_mv = MajorityVote().fit_predict(df_crowd_train)

df_train_mv = pd.merge(df_train, agg_mv, left_on='query', right_on='task')

f1_score(df_train_mv['intent'], df_train_mv['agg_label'], average='macro')

0.9911108888666644

## Dawid-Skene

Let's compare with the more specific probabilistic method called the Dawid-Skene model (DS).

In [ ]:
agg_ds = DawidSkene().fit_predict(df_crowd_train)

df_train_ds = pd.merge(df_train, agg_ds, left_on='query', right_on='task')

f1_score(df_train_ds['intent'], df_train_ds['agg_label'], average='macro')

0.9866875576446533

## GLAD

And finally, let's try one more probabilistic model for aggregation, called GLAD.

In [ ]:
agg_glad_train = GLAD().fit_predict(df_crowd_train)

df_train_glad = pd.merge(df_train, agg_glad_train, left_on='query', right_on='task')

f1_score(df_train_glad['intent'], df_train_glad['agg_label'], average='macro')

0.9933104421553267

As GLAD outperformed other models on `train`, we will use it to aggregate the answers. Let's first look how well it performs on the `test` subset.

In [ ]:
agg_glad_test = GLAD().fit_predict(df_crowd_test)

df_test_glad = pd.merge(df_test, agg_glad_test, left_on='query', right_on='task')

f1_score(df_test_glad['intent'], df_test_glad['agg_label'], average='macro')

0.9850482025641298

# Learning from Crowds

Now let's train a machine learning model based on our crowdsourced data. We will need some boilerplate code for tranforming identifiers of labels and annotators into one-hot vectors as well as preparing data for PyTorch.

In [ ]:
ohe = LabelBinarizer().fit(list(intents))
enc = LabelEncoder().fit(df_crowd_train['worker'].unique())

This class, `CLINC150Dataset`, outputs pairs of $(\text{query}, \text{intent})$ from our dataset, where $\text{intent}$ is the target variable.

In [ ]:
class CLINC150Dataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data

        # We will use the transform() function to embed the query.
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.loc[idx]

        x, y = row['query'], row['intent']

        if self.transform:
            x = self.transform(x)

        return torch.tensor(x), torch.tensor(ohe.transform([y])[0])

This class, `CLINC150CrowdDataset`, outputs triples of $(\text{query}, \text{worker}, \text{intent})$ from our dataset, where $\text{worker}$ is the annotator identifier and $\text{intent}$ is the target variable.

In [ ]:
class CLINC150CrowdDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data

        # We will use the transform() function to embed the query.
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.loc[idx]

        x, w, y = row['task'], row['worker'], row['label']

        if self.transform:
            x = self.transform(x)

        return torch.tensor(x), torch.tensor(enc.transform([w])[0]), torch.tensor(ohe.transform([y])[0], dtype=torch.float32)

We will use the popular [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) sentence-transformers model:

> It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

And load the data, configuring the loaders to use this pre-trained model.

In [ ]:
train_dataset = CLINC150Dataset(df_train, transform=encoder.encode)
crowd_train_dataset = CLINC150CrowdDataset(df_crowd_train, transform=encoder.encode)
test_dataset = CLINC150Dataset(df_test, transform=encoder.encode)
aggregated_dataset = CLINC150Dataset(df_train_glad[['query', 'intent']], transform=encoder.encode)

train_loader = utils.data.DataLoader(train_dataset, batch_size=64)
crowd_train_loader = utils.data.DataLoader(crowd_train_dataset, batch_size=64)
test_loader = utils.data.DataLoader(test_dataset, batch_size=64)
aggregated_loader = utils.data.DataLoader(aggregated_dataset, batch_size=64)

We will use the following model architecture for classification:

1. Input
2. Query embedding using the pre-trained model (`all-MiniLM-L6-v2`, 384 dimensions)
3. Fully-connected layer (384 dimensions → 9 dimensions)
4. Optional during training: CrowdLayer or CoNAL (9 dimensions → 9 dimensions)

We use the cross-entropy loss with five epochs in most cases. During training, CrowdLayer and CoNAL will need annotator identifiers, but these layers are omitted during inference.

In [ ]:
class CLINC150Classifier(L.LightningModule):
    def __init__(self, n_intents, layer=None, n_workers=None):
        super().__init__()

        L.seed_everything(0)

        self.W = nn.Linear(encoder.get_sentence_embedding_dimension(), n_intents)

        self.f1 = torchmetrics.F1Score(task="multiclass", num_classes=n_intents)

        self.layer = layer

        if layer == 'CrowdLayer':
            self.CL = CrowdLayer(n_intents, n_workers, conn_type='mw')
        elif layer == 'CoNAL':
            self.CL = CoNAL(n_intents, n_workers)
        else:
            self.CL = None

    def forward(self, x, w=None):
        emb = x.clone()

        x = self.W(x)

        if self.layer and w is not None:
            if self.layer == 'CrowdLayer':
                x = self.CL(x, w)
            elif self.layer == 'CoNAL':
                x = self.CL(emb, x, w)

        x = x.to(torch.float)

        return x

    def training_step(self, batch, batch_idx):
        if len(batch) == 3:
            x, w, y = batch
            x = self.forward(x, w)
        else:
            x, y = batch
            x = self.forward(x)

        y = y.to(torch.float)

        loss = torch.nn.functional.cross_entropy(x, y)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        self.f1(torch.argmax(self.forward(x), dim=-1), torch.argmax(y, dim=-1))

        self.log('val_f1', self.f1, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

## Training on Aggregated Data

First, we will aggregate our annotated `train` subset using the GLAD model, and evaluate on the `test` ground truth dataset. We will do the same evaluation protocol for all models in our example. Since aggregation reduced our dataset times thrice, we'll use a proportionally higher number of epochs.

In [ ]:
model = CLINC150Classifier(n_intents=len(intents))

trainer = L.Trainer(max_epochs=5 * 3, log_every_n_steps=1)
trainer.fit(model=model, train_dataloaders=aggregated_loader, val_dataloaders=test_loader)

trainer.validate(model, test_loader)

INFO: Global seed set to 0
INFO:lightning.fabric.utilities.seed:Global seed set to 0
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type              | Params
-------------------------------------------
0 | W    | Linear            | 3.5 K 
1 | f1   | MulticlassF1Score | 0     
-------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total p

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_f1           │    0.9777777791023254     │
└───────────────────────────┴───────────────────────────┘

[{'val_f1': 0.9777777791023254}]

## Training on Raw Data

Instead, we can omit the aggregation step and train our model on the entire raw annotated `train` dataset.

In [ ]:
model = CLINC150Classifier(n_intents=len(intents))

trainer = L.Trainer(max_epochs=5, log_every_n_steps=1)
trainer.fit(model=model, train_dataloaders=crowd_train_loader, val_dataloaders=test_loader)

trainer.validate(model, test_loader)

INFO: Global seed set to 0
INFO:lightning.fabric.utilities.seed:Global seed set to 0
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type              | Params
-------------------------------------------
0 | W    | Linear            | 3.5 K 
1 | f1   | MulticlassF1Score | 0     
-------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total p

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_f1           │    0.9740740656852722     │
└───────────────────────────┴───────────────────────────┘

[{'val_f1': 0.9740740656852722}]

## Training with CrowdLayer

CrowdLayer is a method that learns the confusion matrix $\mathbf{A}_w$ of every annotator $w$.

In [ ]:
model = CLINC150Classifier(n_intents=len(intents),
                           layer='CrowdLayer', n_workers=df_crowd_train['worker'].nunique())

trainer = L.Trainer(max_epochs=5, log_every_n_steps=1)
trainer.fit(model=model, train_dataloaders=crowd_train_loader, val_dataloaders=test_loader)
trainer.validate(model, test_loader)

INFO: Global seed set to 0
INFO:lightning.fabric.utilities.seed:Global seed set to 0
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type              | Params
-------------------------------------------
0 | W    | Linear            | 3.5 K 
1 | f1   | MulticlassF1Score | 0     
2 | CL   | CrowdLayer        | 4.5 K 
-------------------------------------------
7.9 K     Trainable params
0         

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_f1           │    0.9740740656852722     │
└───────────────────────────┴───────────────────────────┘

[{'val_f1': 0.9740740656852722}]

## Training with CoNAL

CoNAL is a method that learns annotator-specific confusion matrices $\mathbf{A}_w$ and one common confusion matrix $\mathbf{A}_g$.

In [ ]:
model = CLINC150Classifier(n_intents=len(intents),
                           layer='CoNAL', n_workers=df_crowd_train['worker'].nunique())

trainer = L.Trainer(max_epochs=5, log_every_n_steps=1)
trainer.fit(model=model, train_dataloaders=crowd_train_loader, val_dataloaders=test_loader)
trainer.validate(model, test_loader)

INFO: Global seed set to 0
INFO:lightning.fabric.utilities.seed:Global seed set to 0
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/loc

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_f1           │     0.970370352268219     │
└───────────────────────────┴───────────────────────────┘

[{'val_f1': 0.970370352268219}]

Thank you! This is how one can use [Crowd-Kit](https://github.com/Toloka/crowd-kit) to learn from crowdsourced and noisy-labeled data.